In [3]:
# import libraries
import cv2
import os
from tqdm import tqdm
import numpy as np
import random
from utils import *

In [5]:
# Resize images
def resize_images(input_folder, output_folder, width=2592, height=1944, progress_status=False):
    # Check the status of the input folder and output folder
    if not check_dir(input_folder) and check_dir(output_folder,True):
        print("Please check the input folder and output folder")
        return

    # List all files in the input folder
    files = os.listdir(input_folder)

    # print the output image size
    print(f"Output image size: {width} x {height}")

    if progress_status:
        # Create a progress bar 
        progress_bar = tqdm(total=len(files), desc="Resizing Images")

    for file in files:
        # Check if the file is an image
        # We can add more file types here if we want
        if file.endswith(('.jpg')):
            img = cv2.imread(os.path.join(input_folder, file))

            if img is not None:
                # Get the original image size
                original_size = (img.shape[1], img.shape[0])
                print(f"Original size of {file}: {original_size}")

                # Resize the image
                img = cv2.resize(img, (width, height))

                # Save the resized image to the output folder
                output_path = os.path.join(output_folder, file)
                cv2.imwrite(output_path, img)
        else:
            print(f"Skipping file: {file} (not an image)")
        if progress_status:
            # Update the progress bar
            progress_bar.update(1)

    if progress_status:
        # Close the progress bar when done
        progress_bar.close()

input_folder = "../images/horizon/original"
output_folder = "../images/horizon/resized"
resize_images(input_folder, output_folder)

Output image size: 2592 x 1944
Original size of 1.jpg: (1041, 585)
Original size of 2.jpg: (1041, 585)
Original size of 3.jpg: (1041, 694)
Original size of 4.jpg: (2048, 1365)
Original size of 5.jpg: (2048, 1365)
Original size of 6.jpg: (2048, 1365)
Original size of 7.jpg: (800, 531)
Original size of 8.jpg: (1041, 694)


In [19]:
# Augment images
def augment_images(input_folder, output_folder, num_augmented_images=5, progress_status=False):
    # Check the status of the input folder and output folder
    if not check_dir(input_folder) and check_dir(output_folder,True):
        print("Please check the input folder and output folder")
        return

    # List all files in the input folder
    files = os.listdir(input_folder)

    if progress_status:
        # Create a progress bar 
        progress_bar = tqdm(total=len(files), desc="Augmenting Images")

    for file in files:
        # Check if the file is an image, we can add more file types here if we want
        if file.endswith(('.jpg')):
            # Read the original image using OpenCV
            img = cv2.imread(os.path.join(input_folder, file))

            augmented_images = [img]  # Store the original image as the first one

            # Apply data augmentation techniques multiple times
            for i in range(num_augmented_images):
                augmented_img = img.copy()

                # Randomly adjust brightness
                brightness_factor = random.uniform(0.7, 1.3)
                augmented_img = cv2.convertScaleAbs(augmented_img, alpha=brightness_factor, beta=0)

                # Randomly adjust contrast
                contrast_factor = random.uniform(0.7, 1.3)
                augmented_img = cv2.convertScaleAbs(augmented_img, alpha=contrast_factor, beta=0)

                # Randomly rotate the image without leaving black edges
                angle = random.uniform(-10, 10)
                rows, cols, _ = augmented_img.shape
                M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
                augmented_img = cv2.warpAffine(augmented_img, M, (cols, rows), borderMode=cv2.BORDER_DEFAULT)
                # TODO: rotate needs to be fixed because it may cause some strange effects

                augmented_images.append(augmented_img)

            # Save the augmented images to the output folder
            for idx, augmented_img in enumerate(augmented_images):
                output_filename = f"{os.path.splitext(file)[0]}_augmented_{idx}{os.path.splitext(file)[1]}"
                output_path = os.path.join(output_folder, output_filename)
                cv2.imwrite(output_path, augmented_img)

            if progress_status:
                # Update the progress bar
                progress_bar.update(1)

    if progress_status:
        # Close the progress bar when done
        progress_bar.close()

input_folder = "../images/horizon/resized"
output_folder = "../images/horizon/augmented"
augment_images(input_folder, output_folder, num_augmented_images=5, progress_status=True)


Augmenting Images: 100%|██████████| 8/8 [00:04<00:00,  1.75it/s]
